# CLIP with conformal prediction

Comparing prediciton set size across different datasets

In [1]:
from pathlib import Path
from functools import partial
import pandas as pd
import matplotlib.pyplot as plt; plt.style.use('bmh')
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
assert torch.cuda.is_available()
import open_clip
from tqdm import tqdm
import wilds

## datasets

In [2]:
data_dir = Path('../data')
cifar_train = datasets.CIFAR100(root=data_dir, train=True)
cifar_test = datasets.CIFAR100(root=data_dir, train=False)

In [3]:
cifar_classes = tuple(cifar_test.classes)
cifar_class_map = dict(map(reversed, cifar_test.class_to_idx.items()))

In [4]:
class DS(Dataset):
    def __init__(self, images, labels, transforms=None, target_transforms=None):
        assert len(images) == len(labels)
        self.images = images
        self.labels = labels
        self.transforms = transforms
        self.target_transforms = target_transforms
        
    def __getitem__(self, k):
        x = self.images[k]
        y = self.labels[k]
        if self.transforms is not None:
            x = self.transforms(x)
        if self.target_transforms is not None:
            y = self.target_transforms(y)
        return x, y
    
    def __len__(self):
        return len(self.images)
    

In [5]:
cifar_test_dataset = DS(
    cifar_test.data, cifar_test.targets,
    transforms=transforms.Compose([
        # transforms.ToTensor(),
        # transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
        transforms.ToPILImage(),
    ]), 
)

loader_params = dict(batch_size=16, shuffle=False, pin_memory=True, num_workers=8)
cifar_test_loader = DataLoader(cifar_test_dataset, **loader_params)

In [6]:
wild_dataset.get_subset?

Object `wild_dataset.get_subset` not found.


In [7]:
wild_dataset = wilds.get_dataset(dataset='iwildcam', root_dir=data_dir/'wilds')
wild_test_id = wild_dataset.get_subset('id_test')
wild_test_od = wild_dataset.get_subset('test', frac=0.2)

In [8]:
wild_df = pd.read_csv(wild_dataset._data_dir / 'categories.csv')
wild_class_map = dict(wild_df[['y', 'name']].values)
wild_classes = tuple(wild_class_map.values())

In [9]:
len(wild_test_od), len(wild_test_id)

(8558, 8154)

## CLIP model

In [10]:
tokenizer = open_clip.get_tokenizer('ViT-B-32-quickgelu')
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32-quickgelu', pretrained='laion400m_e32')

In [11]:
cifar_true_class = []
cifar_pred_class = []
cifar_pred_scores = []

with torch.no_grad(), torch.cuda.amp.autocast():
    prompt = tokenizer(['This is an image of a ' + c for c in cifar_classes])
    text_features = model.encode_text(prompt)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    for i in tqdm(range(len(cifar_test_dataset))):
        x, y = cifar_test_dataset[i]
        image_features = model.encode_image(preprocess(x).unsqueeze(0))
        image_features /= image_features.norm(dim=-1, keepdim=True)
        scores = (image_features @ text_features.T).softmax(dim=-1)
        
        cifar_true_class.append(y)
        cifar_pred_class.append(scores.argmax().item())
        cifar_pred_scores.append(scores.detach().cpu().numpy().squeeze())
        
cifar_true_class = np.asarray(cifar_true_class)
cifar_pred_class = np.asarray(cifar_pred_class)
cifar_pred_scores = np.asarray(cifar_pred_scores)

cifar_acc = (cifar_true_class == cifar_pred_class).sum() / cifar_true_class.shape[0]
print(f'CIFAR100 accuracy: {cifar_acc:.1%}')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [14:52<00:00, 11.20it/s]

CIFAR100 accuracy: 68.6%


In [ ]:
wild_id_true_class = []
wild_id_pred_class = []
wild_id_pred_scores = []

with torch.no_grad(), torch.cuda.amp.autocast():
    prompt = tokenizer(['This is an image of a ' + c for c in wild_classes])
    text_features = model.encode_text(prompt)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    for i in tqdm(range(len(wild_test_id))):
        x, y, _ = wild_test_id[i]
        image_features = model.encode_image(preprocess(x).unsqueeze(0))
        image_features /= image_features.norm(dim=-1, keepdim=True)
        scores = (image_features @ text_features.T).softmax(dim=-1)
        
        wild_id_true_class.append(y)
        wild_id_pred_class.append(scores.argmax().item())
        wild_id_pred_scores.append(scores.detach().cpu().numpy().squeeze())
        
wild_id_true_class = np.asarray(wild_id_true_class)
wild_id_pred_class = np.asarray(wild_id_pred_class)
wild_id_pred_scores = np.asarray(wild_id_pred_scores)

wild_od_true_class = []
wild_od_pred_class = []
wild_od_pred_scores = []

with torch.no_grad(), torch.cuda.amp.autocast():
    prompt = tokenizer(['This is an image of a ' + c for c in wild_classes])
    text_features = model.encode_text(prompt)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    for i in tqdm(range(len(wild_test_id))):
        x, y, _ = wild_test_id[i]
        image_features = model.encode_image(preprocess(x).unsqueeze(0))
        image_features /= image_features.norm(dim=-1, keepdim=True)
        scores = (image_features @ text_features.T).softmax(dim=-1)
        
        wild_od_true_class.append(y)
        wild_od_pred_class.append(scores.argmax().item())
        wild_od_pred_scores.append(scores.detach().cpu().numpy().squeeze())
        
wild_od_true_class = np.asarray(wild_od_true_class)
wild_od_pred_class = np.asarray(wild_od_pred_class)
wild_od_pred_scores = np.asarray(wild_od_pred_scores)

wild_id_acc = (wild_id_true_class == wild_id_pred_class).sum() / wild_id_true_class.shape[0]
print(f'WILD ID accuracy: {wild_id_acc:.1%}')
wild_od_acc = (wild_od_true_class == wild_od_pred_class).sum() / wild_od_true_class.shape[0]
print(f'WILD OD accuracy: {wild_od_acc:.1%}')

 40%|████████████████████████████████████████████████████████▉                                                                                    | 3292/8154 [06:51<26:33,  3.05it/s]

## Conformal prediction

In [ ]:
def get_quantile(scores, targets, alpha=0.1):
    n = torch.tensor(targets.size(0))
    score_dist = torch.take_along_dim(1 - scores, targets.unsqueeze(1), 1).flatten()
    qhat = torch.quantile(score_dist, torch.ceil((n + 1) * (1 - alpha)) / n, interpolation="higher")
    return qhat


def make_prediction_sets(scores, qhat, allow_empty_sets=False):
    n = scores.size(0)
    elements_mask = scores >= (1 - qhat)
    if not allow_empty_sets:
        elements_mask[torch.arange(n), scores.argmax(1)] = True
    return elements_mask

def get_coverage(psets, targets, precision=None):
    psets = psets.clone().detach()
    targets = targets.clone().detach()
    n = psets.shape[0]
    coverage = psets[torch.arange(n), targets].float().mean().item()
    if precision is not None:
        coverage = round(coverage, precision)
    return coverage


def get_size(psets, precision=1):
    psets = psets.clone().detach()
    size = psets.sum(1).float().mean().item()
    if precision is not None:
        size = round(size, precision)
    return size


def get_coverage_by_class(psets, targets, num_classes):
    psets = psets.clone().detach()
    targets = targets.clone().detach()
    results = {}
    for c in range(num_classes):
        index = targets == c
        psets_c = psets[index]
        targets_c = targets[index]
        results[c] = get_coverage(psets_c, targets_c)
    return results


def get_efficiency_by_class(psets, targets, num_classes):
    psets = psets.clone().detach()
    targets = targets.clone().detach()
    sizes = psets.sum(1)
    results = {}
    for c in range(num_classes):
        index = targets == c
        psets_c = psets[index]
        results[c] = get_size(psets_c)
    return results


In [ ]:
frac = 0.1
alpha = 0.1

cifar_n = int(round(frac * len(cifar_pred_scores)))
cifar_cal_scores = torch.tensor(cifar_pred_scores[:cifar_n])
cifar_cal_targets = torch.tensor(cifar_true_class[:cifar_n])
cifar_val_scores = torch.tensor(cifar_pred_scores[cifar_n:])
cifar_val_targets = torch.tensor(cifar_true_class[cifar_n:])

cifar_qhat = get_quantile(cifar_cal_scores, cifar_cal_targets, alpha=alpha)
cifar_psets = make_prediction_sets(cifar_val_scores, cifar_qhat)

print(f'CIFAR100 coverage: {get_coverage(cifar_psets, cifar_val_targets):.1%}')
print(f'CIFAR100 set size: {get_size(cifar_psets):.1f}')

wild_id_n = int(round(frac * len(wild_id_pred_scores)))
wild_id_cal_scores = torch.tensor(wild_id_pred_scores[:wild_id_n])
wild_id_cal_targets = torch.tensor(wild_id_true_class[:wild_id_n])
wild_id_val_scores = torch.tensor(wild_id_pred_scores[wild_id_n:])
wild_id_val_targets = torch.tensor(wild_id_true_class[wild_id_n:])

wild_id_qhat = get_quantile(wild_id_cal_scores, wild_id_cal_targets, alpha=alpha)
wild_id_psets = make_prediction_sets(wild_id_val_scores, wild_id_qhat)

print(f'WILD ID coverage: {get_coverage(wild_id_psets, wild_id_val_targets):.1%}')
print(f'WILD ID set size: {get_size(wild_id_psets):.1f}')

wild_od_n = int(round(frac * len(wild_od_pred_scores)))
wild_od_cal_scores = torch.tensor(wild_od_pred_scores[:wild_od_n])
wild_od_cal_targets = torch.tensor(wild_od_true_class[:wild_od_n])
wild_od_val_scores = torch.tensor(wild_od_pred_scores[wild_od_n:])
wild_od_val_targets = torch.tensor(wild_od_true_class[wild_od_n:])

wild_od_qhat = get_quantile(wild_od_cal_scores, wild_od_cal_targets, alpha=alpha)
wild_od_psets = make_prediction_sets(wild_od_val_scores, wild_od_qhat)

print(f'WILD OD coverage: {get_coverage(wild_od_psets, wild_od_val_targets):.1%}')
print(f'WILD OD set size: {get_size(wild_od_psets):.1f}')

In [ ]:
# Fixing random state for reproducibility
np.random.seed(19680801)


# fake data
fs = 10  # fontsize
pos = [1, 2, 4, 5, 7, 8]
data = [np.random.normal(0, std, size=100) for std in pos]

fig, axs = plt.subplots(nrows=2, ncols=5, figsize=(10, 6))

axs[0, 0].violinplot(data, pos, points=20, widths=0.3,
                     showmeans=True, showextrema=True, showmedians=True)
axs[0, 0].set_title('Custom violinplot 1', fontsize=fs)

axs[0, 1].violinplot(data, pos, points=40, widths=0.5,
                     showmeans=True, showextrema=True, showmedians=True,
                     bw_method='silverman')
axs[0, 1].set_title('Custom violinplot 2', fontsize=fs)

axs[0, 2].violinplot(data, pos, points=60, widths=0.7, showmeans=True,
                     showextrema=True, showmedians=True, bw_method=0.5)
axs[0, 2].set_title('Custom violinplot 3', fontsize=fs)

axs[0, 3].violinplot(data, pos, points=60, widths=0.7, showmeans=True,
                     showextrema=True, showmedians=True, bw_method=0.5,
                     quantiles=[[0.1], [], [], [0.175, 0.954], [0.75], [0.25]])
axs[0, 3].set_title('Custom violinplot 4', fontsize=fs)

axs[0, 4].violinplot(data[-1:], pos[-1:], points=60, widths=0.7,
                     showmeans=True, showextrema=True, showmedians=True,
                     quantiles=[0.05, 0.1, 0.8, 0.9], bw_method=0.5)
axs[0, 4].set_title('Custom violinplot 5', fontsize=fs)

axs[1, 0].violinplot(data, pos, points=80, vert=False, widths=0.7,
                     showmeans=True, showextrema=True, showmedians=True)
axs[1, 0].set_title('Custom violinplot 6', fontsize=fs)

axs[1, 1].violinplot(data, pos, points=100, vert=False, widths=0.9,
                     showmeans=True, showextrema=True, showmedians=True,
                     bw_method='silverman')
axs[1, 1].set_title('Custom violinplot 7', fontsize=fs)

axs[1, 2].violinplot(data, pos, points=200, vert=False, widths=1.1,
                     showmeans=True, showextrema=True, showmedians=True,
                     bw_method=0.5)
axs[1, 2].set_title('Custom violinplot 8', fontsize=fs)

axs[1, 3].violinplot(data, pos, points=200, vert=False, widths=1.1,
                     showmeans=True, showextrema=True, showmedians=True,
                     quantiles=[[0.1], [], [], [0.175, 0.954], [0.75], [0.25]],
                     bw_method=0.5)
axs[1, 3].set_title('Custom violinplot 9', fontsize=fs)

axs[1, 4].violinplot(data[-1:], pos[-1:], points=200, vert=False, widths=1.1,
                     showmeans=True, showextrema=True, showmedians=True,
                     quantiles=[0.05, 0.1, 0.8, 0.9], bw_method=0.5)
axs[1, 4].set_title('Custom violinplot 10', fontsize=fs)


for ax in axs.flat:
    ax.set_yticklabels([])

fig.suptitle("Violin Plotting Examples")
fig.subplots_adjust(hspace=0.4)
plt.show()